# Ad Auction Simulator — Quick Tutorial

Welcome to a tiny walkthrough of the `ad-auction-sim` package. We'll meet the main actors: `AdSpot` (the shiny stage), `Bidder` (the hopeful performer), and `Platform` (the strict talent agent).

This notebook is interactive — run the code cells to see the simulator in action.

## 1) Meet the AdSpot — the stage where ads perform

An `AdSpot` represents an ad placement with a number of slots, tags (audience or context), and CTRs (click-through rates) per slot. Think of tags as the crowd: sports fans, cat lovers, or sleepy commuters.

Let's create one!

In [5]:
import os
os.chdir('../')  # Change to the parent directory
from sim.ad_auction import AdSpot

# Create an ad spot with 2 slots, tags, and two position scores
spot = AdSpot(num_slots=2, tags=["sports", "male"], pos=[0.9, 0.5])
print(spot)
print('Number of slots:', spot.num_slots)
print('Tags:', spot.tags)
print('Position Scores:', spot.pos)

Number of slots: 2
Tags: ['sports', 'male']
Position Scores: [0.9, 0.5]


A few quick notes:
- `num_spots` sets how many winners this spot will have (like seats at a tiny ad concert).
- `tags` describe the audience; bidders may value certain tags more.
- `ctrs` are the per-slot multipliers for click rates.

## 2) Meet the Bidder — the ad-hungry competitor

A `Bidder` has a name and a targeting map: how much they value each tag. Higher numbers mean the bidder *really* wants that audience (maybe too much coffee).

Let's instantiate a couple of bidders.

In [6]:
from sim.ad_auction import Bidder

alice = Bidder('Alice', {'sports': 5.0, 'male': 1.0})
bob = Bidder('Bob', {'sports': 3.0, 'tech': 2.0})
charlie = Bidder('Charlie', {'female': 4.0})

print(alice)
print(bob)
print('Alice targeting map:', alice.targeting)

Bidder(Alice)
Bidder(Bob)
Alice targeting map: {'sports': 5.0, 'male': 1.0}


Bidder tips:
- Targeting values can be any float.
- Missing tags are treated as zero when computing valuations.
- You can create many bidders and tweak their bids/valuations for experiments.

You can also have a bidder follow a specific bidding strategy, let's see how!

In [7]:
def underbid(bidder: Bidder, adspot: AdSpot, valuation: float) -> float:
    """A bidding strategy that bids 80% of the true valuation."""
    return 0.8 * valuation

dave = Bidder('Dave', {'sports': 0.6, 'news': 0.4}, bid_func=underbid)

## 3) Platform — the auction maestro

`Platform` accepts a list of `Bidder` objects and runs auctions over `AdSpot`s using different pricing rules (first-price, second-price, GSP). We'll show a tiny valuation function that sums matching targeting weights, and run auctions.

In [8]:
from typing import List
from sim.ad_auction import Platform

def simple_valuation(bidder: Bidder, adspot: AdSpot, ctrs: List[float]) -> float:
    """Compute the bidder's valuation for the adspot by summing the values for each tag.
    
    Notice that missing tags contribute zero.
    
    Args:
        bidder (Bidder): Bidder instance
        adspot (AdSpot): AdSpot instance
        ctrs (List[float]): List of click-through rates (not used in this simple function)
        
    Returns:
        val: The computed valuation for this adspot.
    """
    val = 0.0
    for t in adspot.tags:
        val += bidder.targeting.get(t, 0.0)
    return val

platform = Platform([alice, bob, charlie])
spots = [spot]

for method in ['first_price', 'second_price', 'gsp']:
    print(f"Method: {method}")
    results = platform.assign(spots, method=method, valuation_fn=simple_valuation)
    for i, r in enumerate(results):
        print(f"AdSpot {i}: winners={r['winners']}, prices={r['prices']}")

Method: first_price
AdSpot 0: winners=[Bidder(Bob), Bidder(Alice)], prices=[3.0, 6.0]
Method: second_price
AdSpot 0: winners=[Bidder(Bob), Bidder(Alice)], prices=[6.0, 0.0]
Method: gsp
AdSpot 0: winners=[Bidder(Alice), Bidder(Bob)], prices=[3.0, 0.0]


What the `assign` output contains:
- `winners`: list of bidder names that won each slot (ordered by slot).
- `prices`: list of prices charged per slot.

Try changing bidder targeting values or ad spot tags to see different winners.

For example, we'll try adding Dave to the experiment, while keeping the same adspots.

In [9]:
platform.add_bidder(dave)

for method in ['first_price', 'second_price', 'gsp']:
    print(f"Method: {method}")
    results = platform.assign(spots, method=method, valuation_fn=simple_valuation)
    for i, r in enumerate(results):
        print(f"AdSpot {i}: winners={r['winners']}, prices={r['prices']}")

Method: first_price
AdSpot 0: winners=[Bidder(Alice), Bidder(Bob)], prices=[6.0, 3.0]
Method: second_price
AdSpot 0: winners=[Bidder(Bob), Bidder(Alice)], prices=[6.0, 0.48]
Method: gsp
AdSpot 0: winners=[Bidder(Alice), Bidder(Bob)], prices=[3.0, 0.48]


For the experiment shown in class, go to the `experiments` folder

---
Notes: This tutorial uses the code from the `sim` package in this repository. If imports fail, ensure your notebook kernel's working directory is the repository root or install the package into the environment (e.g., `pip install -e .`).

That's all — enjoy the ad auction circus! 🤹‍♀️